# Test the self hosted service 
The service is set up with docker. To find out how to change the possibilities what to control the possible routing parameters from the outside in this notbook a call for the service is implemented. 

In [1]:
import pandas as pd
from routingpy import ORS, Google
from routingpy.exceptions import RouterApiError
import folium

In [25]:
class RoutingTool: 
    """Calculate different routes for each given start and end point."""
    def __init__(self):
        '''Define url.'''
        self.base_url_ors = 'http://localhost:8080/ors'
        self.setting_ors_cargo_bike()
        
    def setting_ors_cargo_bike(self, elevation = True, avoid_features = ['junction', 'steps'], steepness_difficulty = 0):

        ors_cargo_bike_green_args = {
            'elevation': elevation, # take elevation into account
            'language': 'de',
            'options': {
                'avoid_features': avoid_features, # avoid steps
                'profile_params': {
                    'weightings': {
                        'steepness_difficulty': steepness_difficulty, # Beginner steepness difficulty
                        'green': 1
                    },
                },
            },
            'extra_info': [
                'waytype',
                'surface'
            ],
        }     
        ors_cargo_bike_args = {
            'elevation': elevation, # take elevation into account
            'language': 'de',
            'options': {
                'avoid_features': avoid_features, # avoid steps
                'profile_params': {
                    'weightings': {
                        'steepness_difficulty': steepness_difficulty, # Beginner steepness difficulty
                    },
                },
            },
            'extra_info': [
                'waytype',
                'surface'
            ],
        }   
        self.args = [ors_cargo_bike_green_args, ors_cargo_bike_args]

    def calculate_directions(self, coordinates):
        routes = []
        client, profile, args, name = ORS(base_url = self.base_url_ors), 'cycling-cargo', self.args, 'ORS'
        try:
            for arg in args: 
                print("Trying directions: ", name)
                route = client.directions(locations=coordinates, profile=profile, **arg)
                print("Successful")
                routes.append((route, name))
        except RouterApiError as rae:
            print("Error", rae.message['error'])
            self.rae = rae
            return False
        except KeyError as ke:
            print("Error", ke.message['error'])
            return False
        self.routes = routes
        return True
    def direction_wgs84(self):
        '''Return the coordinates in wgs84.'''
        return self.routes
    def direction_etrs89(self):
        '''Return the coordinates in etrs89.'''
        try:
            ors_coords = [self.convert_coord_systems(geom) for geom in self.routes[0][0].geometry]
        except:
            return False
        etrs89  = ors_coords
        return etrs89
        


In [24]:
routes = []
coord = [[9.17789, 48.78291], [9.1809,48.7873]] 
tool_track = RoutingTool()
if tool_track.calculate_directions(coord):
    routes.append(tool_track.routes[0][0])
    routes.append(tool_track.routes[1][0])

Trying directions:  ORS
Error {'code': 2003, 'message': "Parameter 'avoid_features' has incorrect value of 'junctions'."}


In [4]:
class DisplayRoute():
    def swap_lat_lon(self,coordinates):
        return list(map(lambda point: [point[1], point[0]], coordinates))
    def display_map(self,routes):
        results = self.setting_up_api(routes)
        my_map = folium.Map()

        for route, name, color in results:
            points = self.swap_lat_lon(route.geometry) 
            feature_group = folium.FeatureGroup(name=name)
            folium.PolyLine(points, color=color, opacity=0.5, tooltip=name).add_to(feature_group)
            feature_group.add_to(my_map)

        folium.LayerControl(collapsed=False, hideSingleBase=True).add_to(my_map)
        my_map.fit_bounds(my_map.get_bounds())
        return my_map
    def setting_up_api(self, routes):
        settings = []
        settings.append((routes[0], 'Green', 'green'))
        settings.append((routes[1], ' Not Green', 'blue'))
        return settings

In [5]:
map_routes = DisplayRoute()
map_routes.display_map(routes)

IndexError: list index out of range